<a href="https://colab.research.google.com/github/yeongju98/2022_1_DA/blob/main/Hyperparameter_Tuning_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning 과제

- Manual Search
- Random Search (Grid Search 참고)
- Bayesian Optimization using HyperOpt
<br>연습해보기!

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import load_iris
import warnings
warnings.filterwarnings('ignore')
import time
from functools import partial

In [2]:
iris = load_iris()

X = iris.data
y = iris.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

## 1. Manual Search
사이킷런 Logistic Regresstion https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

In [4]:
'''
maunal 1 
'''

manual1 = LogisticRegression( random_state=100)

manual1.fit(X_train,y_train)
y_pred = manual1.predict(X_train)

# 정확도를 계산해주세요
print('정확도: {}'.format(manual1.score(X_train, y_train)))


정확도: 0.9642857142857143


In [5]:
'''
maunal 2 
'''
# 다양한 하이퍼파라미터 값을 사용하여 또 다른 manual search를 해보세요
# manual1 에 비해 정확도가 더 높아지면 더 좋습니다 !

manual2 = LogisticRegression(solver='saga')

manual2.fit(X_train,y_train)
y_pred = manual2.predict(X_train)

# 정확도를 계산해주세요
print('정확도: {}'.format(manual2.score(X_train, y_train)))


정확도: 0.9821428571428571


## 2. Grid Search

In [6]:
from sklearn.model_selection import GridSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 그리드 서치 호출
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
grid_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 3.915372848510742


## 3. Random Search
grid search 코드와 매우 유사하니 참고해서 빈칸 채워주세요 !

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

# 파라미터 후보 설정
random = dict()
random['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
random['penalty'] = ['none', 'l1', 'l2', 'elasticnet']


# 그리드 서치 호출
model = LogisticRegression()
random_search = RandomizedSearchCV(estimator = model,param_distributions = random, n_iter = 10, 
                               cv = 4, verbose= 1, random_state= 101, n_jobs = -1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

# 모델 적합
random_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {random_search.score(X_test, y_test)}")

print(f"best parameters: {random_search.best_params_}")
print(f"best CV score: {random_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
test set score : 1.0
best parameters: {'solver': 'lbfgs', 'penalty': 'l2'}
best CV score: 0.9553571428571429
time : 8.14478087425232


## 4. HyperOpt

In [8]:
!pip install hyperopt

In [57]:
import hyperopt

In [29]:
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK,Trials
from sklearn.model_selection import cross_val_score

In [75]:


space = {
    'penalty' : hp.choice('penalty', ['l2', 'none']), 
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'max_iter' : hp.choice('max_iter',[100,1000])
}

def objective(space):
    model = LogisticRegression(penalty=space['penalty'], fit_intercept = space['fit_intercept'], 
                               max_iter=space['max_iter'])
    accuracy = cross_val_score(model, X_train, y_train, cv = 4).mean()
    return {'loss': -accuracy, 'status': STATUS_OK }





In [76]:
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20, 
            trials= trials)

print(best)


100%|██████████| 20/20 [00:02<00:00,  8.81it/s, best loss: -0.9553571428571429]
{'fit_intercept': 1, 'max_iter': 0, 'penalty': 1}


In [77]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

pen = {0: 'l1', 1: 'l2', 2: 'elasticnet', 3: None}
incpt = {0: True, 1: False}

trained = LogisticRegression(penalty = pen[best['penalty']],
                             fit_intercept = incpt[best['fit_intercept']],
                             max_iter = best['max_iter']).fit(X_train,y_train)

# 정확도 도출 부분까지 코드 짜시면 됩니다!
obj_pf = trained.predict(X_test)
print(confusion_matrix(y_test,obj_pf))
print(classification_report(y_test,obj_pf))
obj_acc = accuracy_score(y_test,obj_pf)



[[ 0  0 16]
 [ 0  0 11]
 [ 0  0 11]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        11
           2       0.29      1.00      0.45        11

    accuracy                           0.29        38
   macro avg       0.10      0.33      0.15        38
weighted avg       0.08      0.29      0.13        38



## 선택) Bayesian Optimization 사용


In [ ]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization